In [1]:
!pip install -q transformers datasets peft trl accelerate bitsandbytes huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip uninstall -y datasets fsspec gcsfs

!pip install fsspec==2025.3.0
!pip install gcsfs==2025.3.2
!pip install datasets==3.6.0


Found existing installation: datasets 3.6.0
Uninstalling datasets-3.6.0:
  Successfully uninstalled datasets-3.6.0
Found existing installation: fsspec 2025.3.0
Uninstalling fsspec-2025.3.0:
  Successfully uninstalled fsspec-2025.3.0
Found existing installation: gcsfs 2025.3.2
Uninstalling gcsfs-2025.3.2:
  Successfully uninstalled gcsfs-2025.3.2
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.4.0 requires gcsfs>=2023.3.0, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Using cached datasets-3.6.0-py3-none-any.whl.met

In [3]:
from huggingface_hub import login
login(token="")

In [4]:
from google.colab import files

# Prompt file upload
uploaded = files.upload()

# Optional: check the file
for fn in uploaded.keys():
    print(f"Uploaded file: {fn}")

Saving cece_autodataset.jsonl to cece_autodataset.jsonl
Uploaded file: cece_autodataset.jsonl


In [5]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer

def format_chat(example):
    dialogue = ""
    for turn in example["messages"]:
        role = turn["role"]
        content = turn["content"]
        if role == "system":
            dialogue += f"System: {content}\n"
        elif role == "user":
            dialogue += f"User: {content}\n"
        elif role == "assistant":
            dialogue += f"Assistant: {content}\n"
    return {"text": dialogue}

# Load your JSONL dataset
dataset = load_dataset("json", data_files="cece_autodataset.jsonl", split="train")
dataset = dataset.map(format_chat)

# Load DialoGPT-small
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Important!

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,  # LoRA + memory efficient
    device_map="auto"
)

# Set up LoRA config
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, peft_config)

# Train the model
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=TrainingArguments(
        output_dir="./cece-chatbot",
        per_device_train_batch_size=2,
        num_train_epochs=3,
        save_total_limit=1,
        logging_dir="./logs",
        logging_steps=10,
        report_to="none",
    ),
)
trainer.train()

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

Tokenizing train dataset:   0%|          | 0/600 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/600 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:463: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
10,2.664000
20,2.025200
30,2.172400
40,1.849100
50,1.674900
60,1.836800
70,1.696300
80,1.707900
90,1.473500
100,1.531600


TrainOutput(global_step=900, training_loss=0.6595106213622624, metrics={'train_runtime': 263.1056, 'train_samples_per_second': 6.841, 'train_steps_per_second': 3.421, 'total_flos': 727204841349120.0, 'train_loss': 0.6595106213622624})

In [8]:
# Save the trained model to disk
trainer.save_model("./cece-chatbot")
tokenizer.save_pretrained("./cece-chatbot")

# Then push
model.push_to_hub("ccm32004/cece-chatbot")
tokenizer.push_to_hub("ccm32004/cece-chatbot")

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/ccm32004/cece-chatbot/commit/dbc207b3ef164ed6b5b32862da226d8d09bb65f6', commit_message='Upload tokenizer', commit_description='', oid='dbc207b3ef164ed6b5b32862da226d8d09bb65f6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ccm32004/cece-chatbot', endpoint='https://huggingface.co', repo_type='model', repo_id='ccm32004/cece-chatbot'), pr_revision=None, pr_num=None)

In [9]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.7 MB/s eta 0:00:00


In [13]:
import gradio as gr
from transformers import pipeline

system_prompt = """
You are CeceGPT, a chatbot trained exclusively on Cece's personal dataset. You are NOT allowed to make up facts, opinions, or guesses.
Only answer questions based on Cece’s known experiences, projects, skills, and preferences as described in the fine-tuning data.
If a question goes beyond what you were trained on, respond with: “I don’t have that information.”
"""

tokenizer = AutoTokenizer.from_pretrained("ccm32004/cece-chatbot")
chat = pipeline("text-generation", model="ccm32004/cece-chatbot", tokenizer=tokenizer)

def respond(message, history):
    prompt = tokenizer.apply_chat_template(
        [{"role": "system", "content": system_prompt},
         {"role": "user", "content": message}],
        tokenize=False,
        add_generation_prompt=True
    )

    result = chat(prompt, max_new_tokens=100, pad_token_id=tokenizer.pad_token_id)[0]['generated_text']
    output = result[len(prompt):].strip()

    # ✅ Truncate hallucinated turns
    for stop in ["\nUser:", "\nAssistant:", "\nSystem:", "\n\n"]:
        if stop in output:
            output = output.split(stop)[0].strip()
            break

    return output

gr.ChatInterface(respond).launch()


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c4801c243367d3bf4e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
